![title](data/logo.png)
#### LAZ File Reading, Writing and Plotting 

1. Read LAZ files from disk using PDAL and extract spatial boundary information.
3. Graph data and create visualizations of the point clouds using PYLAS.

#### Load pdal and other dependencies

In [11]:
import pdal
import json
import numpy as np
import pandas as pd
import subprocess
import os
import matplotlib
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
matplotlib.use('Agg')

https://pdal.io/python.html#python  
conda install python-pdal

In [2]:
# Always nice to check that you're in the right place
%pwd

'C:\\Users\\niko8816\\Projects\\laz'

In [3]:
# Load in the LAZ file
lasfile = 'data\SNEX19_TLS_PC_CRREL_09232019_FL2D_WGS84.laz'

pipeline = [
        {
            "type": "readers.las",
            "filename": lasfile
        },
        {
            "type": "filters.sort",
            "dimension": "Z"
        }
    ]

pipeline = pdal.Pipeline(json.dumps(pipeline))
pipeline.execute()
arrays = pipeline.arrays

40028042

In [4]:
# Prints a point count of the data
print('Point count:', json.loads(pipeline.metadata)["metadata"]["readers.las"]["count"])

Point count: 40028042


In [5]:
# Prints the .las file schema, or hierarchy
pipeline.schema

{'schema': {'dimensions': [{'name': 'X', 'size': 8, 'type': 'floating'},
   {'name': 'Y', 'size': 8, 'type': 'floating'},
   {'name': 'Z', 'size': 8, 'type': 'floating'},
   {'name': 'Intensity', 'size': 2, 'type': 'unsigned'},
   {'name': 'ReturnNumber', 'size': 1, 'type': 'unsigned'},
   {'name': 'NumberOfReturns', 'size': 1, 'type': 'unsigned'},
   {'name': 'ScanDirectionFlag', 'size': 1, 'type': 'unsigned'},
   {'name': 'EdgeOfFlightLine', 'size': 1, 'type': 'unsigned'},
   {'name': 'Classification', 'size': 1, 'type': 'unsigned'},
   {'name': 'ScanAngleRank', 'size': 4, 'type': 'floating'},
   {'name': 'UserData', 'size': 1, 'type': 'unsigned'},
   {'name': 'PointSourceId', 'size': 2, 'type': 'unsigned'},
   {'name': 'GpsTime', 'size': 8, 'type': 'floating'}]}}

In [6]:
# Print a very long, but useful list of metadata
json.loads(pipeline.metadata)["metadata"]["readers.las"]

{'comp_spatialreference': 'COMPD_CS["unknown",PROJCS["WGS 84 / UTM zone 12N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-111],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32612"]],VERT_CS["unknown",VERT_DATUM["World Geodetic System 1984",2005,AUTHORITY["EPSG","6326"]],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Up",UP]]]',
 'compressed': True,
 'count': 40028042,
 'creation_doy': 364,
 'creation_year': 2019,
 'dataformat_id': 1,
 'dataoffset': 429,
 'filesource_id': 0,
 'global_encoding': 0,
 'global_encoding_base64': 'AAA=',

In [7]:
# This subprocess is necessary to extract -info, which contains spatial information using python
result = subprocess.run(['pdal', 'info', lasfile],
                        stderr = subprocess.PIPE,  # stderr and stdout get
                        stdout = subprocess.PIPE)  # captured as bytestrings

json_result = json.loads(result.stdout.decode())

In [8]:
json_result['stats']['bbox']['native']

{'bbox': {'maxx': 744327.366,
  'maxy': 4324194.645,
  'maxz': 3106.058999,
  'minx': 744174.899,
  'miny': 4324076.375,
  'minz': 3074.013},
 'boundary': {'type': 'Polygon',
  'coordinates': [[[744174.898998296, 4324076.375000028, 3074.013000006031],
    [744174.898998296, 4324194.644996615, 3074.013000006031],
    [744327.365997279, 4324194.644996615, 3106.0589994584807],
    [744327.365997279, 4324076.375000028, 3106.0589994584807],
    [744174.898998296, 4324076.375000028, 3074.013000006031]]]}}

In [9]:
print('Max X:', json.loads(pipeline.metadata)["metadata"]["readers.las"]["maxx"], 'Min X:', json.loads(pipeline.metadata)["metadata"]["readers.las"]["minx"])

Max X: 744327.366 Min X: 744174.899


### Make a histogram from the LAZ data  
Get a visual summary of your data

In [12]:
def make_plot(dimensions, filename, dpi=300):
    figure_position = 1
    row = 1
    fig = plt.figure(figure_position, figsize=(6, 8.5), dpi=dpi)
    keys = dimensions.dtype.fields.keys()

    for key in keys:
        dimension = dimensions[key]
        ax = fig.add_subplot(len(keys), 1, row)

        n, bins, patches = ax.hist( dimension, 30,
                                    #normed=0,
                                    facecolor='grey',
                                    alpha=0.75,
                                    align='mid',
                                    histtype='stepfilled',
                                    linewidth=None)

        ax.set_ylabel(key, size=10, rotation='horizontal')
        ax.get_xaxis().set_visible(False)
        ax.set_yticklabels('')
        ax.set_yticks((),)
        ax.set_xlim(min(dimension), max(dimension))
        ax.set_ylim(min(n), max(n))
        row = row + 1
        figure_position = figure_position + 1
    output = BytesIO()
    plt.savefig(output,format="PNG")

    o = open(filename, 'wb')
    o.write(output.getvalue())
    o.close()
    return True

make_plot(arrays[0], 'histogram.png')

NameError: name 'arrays' is not defined

### Import and plot data using pylas  
Note: this method currently distorts values but plots data correctly

In [ ]:
#pylas dependencies
#pip install pylas[lazrs]
import pylas
import lazrs
import s3fs
from laspy.file import File
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
#!LAStools/bin/laszip -i C:/Users/niko8816/Desktop/Data/SnowEX/SNEX20_TLS_CRREL_Sample/SNEX19_TLS_PC_CRREL_09232019_FL2D_WGS84.laz -o CRREL.las

In [ ]:
sample_data = './data/CRREL.las'
export_path = 'export//'

In [ ]:
inFile = File(sample_data, mode='r')

In [ ]:
df = pd.DataFrame() 
df['X'] = inFile.X 
df['Y'] = inFile.Y 
df['Z'] = inFile.Z
df['intensity'] = inFile.intensity
display(df)

In [ ]:
cvs = ds.Canvas(plot_width=1000, plot_height=1000)
agg = cvs.points(df, 'X', 'Y', ds.mean('Z'))
img = tf.shade(agg)#, cmap=['lightblue', 'darkblue'], how='log')
tf.set_background(tf.shade(agg, cmap=cm.inferno),"black")